In [ ]:
# By default, available from 8AM-7:20PM ET

# Unavailable time given the member
# Chairs 
# C1 The University of British Columbia M/W 10 to 12
# C2 University of Calgary T/Th 2 to 4
# C3 National Research Council Canada M/W 1 o 3
# C4 York University T/F 9 to 11
# C5 University of California – San Diego Th 8 to 12
# C6 Université de Sherbrooke W/F 10 to 12
# C7 University of Victoria T/Th 1 to 3
# C8 Université Laval Th 8 to 12
# C9 Hydro-Québec T/F 10 to 12
# C10 Canadian Food Inspection Agency M/F 11 to 2





In [ ]:



# Members 
# M1 University of Toronto T/Th 2 to 4
# M2 Toronto Metropolitan University M/W 1 o 3
# M3 University of Michigan T/F 9 to 11
# M4 École Polytechnique de Montréal M/Th 11 to 1
# M5 University of Guelph M/F 2 to 4
# M6 University of Calgary M/F 3 to 5
# M7 University of Prince Edward Island M 1 to 3
# M8 University of Manitoba M/W 10 to 12
# M9 National Research Council Th 8 to 12
# M10 Royal Military College of Canada F 11 to 1
# M11 Dalhousie University M/Th 11 to 1
# M12 University of Windsor M/F 2 to 4
# M13 Queen’s University M/F 3 to 5
# M14 The University of Western Ontario W 12 to 5
# M15 École Polytechnique de Montréal W/F 10 to 12
# M16 Université du Québec en Abitibi-Témiscamingue T/Th 1 to 3
# M17 Université du Québec à Trois Rivieres M/W 10 to 12
# M18 FPrimeC Solutions Inc. T/F 10 to 12
# M19 Institut national de la recherche scientifique Thu 10 to 3
# M20 Université du Québec à Chicoutimi T/Th 3 to 5
# M21 Université Catholique de Louvain M/T 1 to 3
# M22 University of Northern British Columbia Th 8 to 12
# M23 University of Saskatchewan M/T 10 to 12
# M24 University of Alberta T/W 3 to 5
# M25 University of Waterloo T/W 3 to 5
# M26 University of Alberta M/W 10 to 12
# M27 Brock University M/F 11 to 2
# M28 University of Cyprus T/Th 1 to 3
# M29 University of Manitoba F 12 to 4
# M30 McMaster University T/Th 8 to 10
